In [1]:
import pyam
import pandas as pd

import dotenv
import os
from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
dotenv.load_dotenv()

True

Step 1: Read in the Gidden and Gasser et al. dataset

In [3]:
df = pyam.IamDataFrame(
    Path(
        os.environ['GIDDEN_DATA']
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file /Users/gauravganti/Documents/github_projects/2023_egr_ch7_equity/data/gidden_et_al_2022_ar6_reanalysis_data.csv


Step 2: Read in the Gidden and Gasser et al. metadata file

In [4]:
reanalysis_meta = pd.read_csv(
    Path(
        os.environ['GIDDEN_META']
    ),
    index_col=[0,1]
)

Step 3: Read in the Schleussner et al. metadata file

In [5]:
schleussner_meta = pd.read_excel(
    Path(
        os.environ['SCHLEUSSNER_META']
    ),
    index_col=[0,1]
)

Step 4: Downselect the following elements from the Schleussner et al metadata:
* The models and scenarios that are also present in the Gidden et al. metadata
* The column 'category_new_rolling_mean'

In [6]:
schleussner_meta_downselected = (
    schleussner_meta
    .loc[reanalysis_meta.index, 'category_new_rolling_mean']
)

Step 5: Assign this as an additional column to the Gidden et al. metadata

In [7]:
reanalysis_meta.loc[:, 'category_new_rolling_mean'] = schleussner_meta_downselected

Step 6: Assign this to the metadata of the dataset we loaded in Step 1.

In [8]:
df.meta = reanalysis_meta

Step 7: Filter out for the 1.5°C scenarios.

In [9]:
df_1p5 = (
    df
    .filter(
        Category=['C1','C2']
    )
)

Step 8: Calculate the gross emission reductions.

In [10]:
df_1p5.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total',
    components=[
        'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct',
        'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land'
    ],
    append=True
)

In [11]:
df_1p5.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
    components=[
        'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
        'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total' # Checked with Matt
    ],
    append=True
)

Step 9: Convert all units to Gt

In [12]:
df_1p5.unit_mapping

{'AR6 Reanalysis|OSCARv3.2|Carbon Removal': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases': 'Mt CO2-equiv/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only': 'Mt CO2-equiv/yr'}

In [13]:
df_1p5.convert_unit(
    current='Mt CO2/yr',
    to='Gt CO2/yr',
    inplace=True
)

In [14]:
df_1p5.convert_unit(
    current='Mt CO2-equiv/yr',
    to='Gt CO2-equiv/yr',
    inplace=True
)

In [15]:
df_1p5.unit_mapping

{'AR6 Reanalysis|OSCARv3.2|Carbon Removal': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases': 'Gt CO2-equiv/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only': 'Gt CO2-equiv/yr'}

Step 10: Read in the "World" AR6 data to get the values for the `Emissions|CH4` variable

In [16]:
ar6_data = pd.read_csv(
    Path(
        os.environ['AR6_DATA']
    ),
    index_col=[0,1]
)

Step 11: First filter the CH4 emissions from the dataset.

In [17]:
ar6_ch4_data = ar6_data[ar6_data['Variable']=='AR6 climate diagnostics|Infilled|Emissions|CH4']

Step 12: Now filter for the scenarios assessed by Gidden and Gasser et al. and then concatenate to the main dataframe.

In [18]:
ar6_ch4_data_selected = pyam.IamDataFrame(
    ar6_ch4_data
    .loc[df_1p5.meta.index, :]
    .reset_index()
)

In [19]:
compiled_1p5_data = pyam.concat(
    [
        df_1p5,
        ar6_ch4_data_selected
    ]
)

Step 13: Now, we want to add the 20 year rate of reduction in methane emissions to the metadata.

In [20]:
data_ch4 = (
    compiled_1p5_data
    .filter(
        variable='*Emissions|CH4'
    )
    .timeseries()
    .reset_index()
    .set_index(['model', 'scenario'])
)

In [21]:
column_net_zero = 'Year of netzero CO2 emissions (Harm-Infilled) table'

In [22]:
custom_scens = []
for mod, scen in compiled_1p5_data.meta.index:
    net_zero_time = compiled_1p5_data.meta.loc[(mod, scen), column_net_zero]
    time_minus_20 = net_zero_time - 20
    # Now we calculate the difference for this scenario
    try:
        diff_ch4 = (
            data_ch4.loc[(mod, scen), net_zero_time]
            -
            data_ch4.loc[(mod, scen), time_minus_20]
        ) / 20
    except:
        diff_ch4 = (
            data_ch4.loc[(mod, scen), net_zero_time]
            -
            data_ch4.loc[(mod, scen), net_zero_time-10]
        ) / 10
        custom_scens.append((mod, scen))
    # Assign it to a metadata column
    compiled_1p5_data.meta.loc[(mod, scen), 'ch4_reduction'] = diff_ch4

Step 14: We will need to calculate the cumulative removals between net zero and 2100

In [23]:
cdr_total = (
    compiled_1p5_data
    .filter(
        variable='*Total',
        region='World'
    )
    .timeseries()
)

In [24]:
cdr_cumulative_net_zero_2100 = cdr_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=compiled_1p5_data.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=2100
    ),
    axis=1
)

In [25]:
compiled_1p5_data.set_meta(
    meta=cdr_cumulative_net_zero_2100,
    name='cumulative_cdr_netzero_2100'
)

Step 15: Set the gross emissions and total CDR at net zero across all the pathways

In [26]:
cdr_net_zero = cdr_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=compiled_1p5_data.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=compiled_1p5_data.meta.loc[(x.name[0], x.name[1]), column_net_zero]
    ),
    axis=1
)

In [27]:
compiled_1p5_data.set_meta(
    meta=cdr_net_zero,
    name='cdr_net_zero'
)

In [28]:
gross_total = (
    compiled_1p5_data
    .filter(
        variable='*Gross',
        region='World'
    )
    .timeseries()
)

In [29]:
gross_net_zero = gross_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=compiled_1p5_data.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=compiled_1p5_data.meta.loc[(x.name[0], x.name[1]), column_net_zero]
    ),
    axis=1
)

In [30]:
compiled_1p5_data.set_meta(
    meta=gross_net_zero,
    name='gross_net_zero'
)

In [31]:
gross_cumulative = gross_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=compiled_1p5_data.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=2100
    ),
    axis=1
)

In [32]:
compiled_1p5_data.set_meta(
    meta=gross_cumulative,
    name='cumulative_gross_netzero_2100'
)

Step 15: Save this data out for further analysis.

In [33]:
compiled_1p5_data.to_excel(
    Path(
        '../data/101_data.xlsx'
    )
)